In [1]:
import requests
import json
import pandas as pd
from time import sleep

In [2]:
df = pd.read_csv('../output/codici_umbria_er_20241117.csv', dtype=str)
df_comuni = df[df['tipo'] == 'CM']
# Filtering for only Emilia Romagna
df_er = df_comuni[df_comuni['cod_regione'] == '08']

In [3]:
BASE_URL = 'https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/'

df_er['url_aff'] = df_er.apply(
    lambda row: f"{BASE_URL}{row['cod_regione']}/PR/{row['cod_provincia_2']}/CM/{row['cod_comune']}", axis=1
)

df_er['url'] = df_er.apply(
    lambda row: f"https://elezioni.interno.gov.it/risultati/20241117/regionali/scrutini/italia/{row['cod_regione']}{row['cod_provincia']}{row['cod_comune']}", axis=1
)
df_er.sample(3)

/tmp/ipykernel_1884/3372484651.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_er['url_aff'] = df_er.apply(
/tmp/ipykernel_1884/3372484651.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_er['url'] = df_er.apply(


,cod,desc,tipo,tipo_comune,dt_agg,cod_ele,cod_regione,regione,cod_provincia,provincia,cod_provincia_2,cod_comune,url_aff,url
19,080130130170,CASTELLO D'ARGILE,CM,N,NaN,7,08,EMILIA-ROMAGNA,013,BOLOGNA,013,0170,https://eleapi.interno.gov.it/siel/PX/scrutini...,https://elezioni.interno.gov.it/risultati/2024...
138,080500500240,MONTECRETO,CM,N,NaN,7,08,EMILIA-ROMAGNA,050,MODENA,050,0240,https://eleapi.interno.gov.it/siel/PX/scrutini...,https://elezioni.interno.gov.it/risultati/2024...
70,080290290110,LAGOSANTO,CM,N,NaN,7,08,EMILIA-ROMAGNA,029,FERRARA,029,0110,https://eleapi.interno.gov.it/siel/PX/scrutini...,https://elezioni.interno.gov.it/risultati/2024...


In [4]:
headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'en-US,en;q=0.6',
    'dnt': '1',
    'origin': 'https://elezioni.interno.gov.it',
    'priority': 'u=1, i',
    'referer': 'https://elezioni.interno.gov.it/',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'sec-gpc': '1'
}

In [5]:
comuni_data = []

for idx, url in enumerate(df_er['url_aff'][:2]):
   print(f"\nProcessing record {idx + 1}")
   print(f"URL: {url}")

   response = requests.get(url, headers=headers)
   print(f"Response status code: {response.status_code}")
   
   data = response.json()

   comune = df_er.iloc[idx]['desc']
   provincia = df_er.iloc[idx]['provincia']
   regione = df_er.iloc[idx]['regione']
   url = df_er.iloc[idx]['url']

   print(f"Processing data for Comune: {comune}, Provincia: {provincia}")

   # Start with basic comune info
   comune_info = {
       'comune': comune,
       'provincia': provincia,
       'regione': regione,
       'url': url
   }
   
   # Add candidate totals
   if 'cand' in data:
       for cand in data['cand']:
           nome_completo = cand['nome'] + " " + cand['cogn']
           comune_info[f"{nome_completo}_total"] = cand['voti']
           comune_info[f"{nome_completo}_perc"] = cand['perc']
           print(f"{nome_completo}: {cand['voti']} votes ({cand['perc']}%)")

   comuni_data.append(comune_info)
   
   print(f"Processed {comune}")
   sleep(0.1)

# Create DataFrame
print("\nCreating DataFrame...")
df_data = pd.DataFrame(comuni_data)

print("\nSample of processed data:")
print(df_data.head())


Processing record 1
URL: https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/08/PR/013/CM/0005


Response status code: 200
Processing data for Comune: ALTO RENO TERME, Provincia: BOLOGNA
MICHELE DE PASCALE: 1123 votes (48,93%)
ELENA UGOLINI: 1095 votes (47,71%)
FEDERICO SERRA: 57 votes (2,48%)
LUCA TEODORI: 20 votes (0,87%)
Processed ALTO RENO TERME

Processing record 2
URL: https://eleapi.interno.gov.it/siel/PX/scrutiniR/DE/20241117/TE/07/RE/08/PR/013/CM/0010


Response status code: 200
Processing data for Comune: ANZOLA DELL'EMILIA, Provincia: BOLOGNA
MICHELE DE PASCALE: 2992 votes (65,77%)
ELENA UGOLINI: 1409 votes (30,97%)
FEDERICO SERRA: 91 votes (2,00%)
LUCA TEODORI: 57 votes (1,25%)
Processed ANZOLA DELL'EMILIA

Creating DataFrame...

Sample of processed data:
               comune provincia         regione  \
0     ALTO RENO TERME   BOLOGNA  EMILIA-ROMAGNA   
1  ANZOLA DELL'EMILIA   BOLOGNA  EMILIA-ROMAGNA   

                                                 url  \
0  https://elezioni.interno.gov.it/risultati/2024...   
1  https://elezioni.interno.gov.it/risultati/2024...   

   MICHELE DE PASCALE_total MICHELE DE PASCALE_perc  ELENA UGOLINI_total  \
0                      1123                   48,93                 1095   
1                      2992                   65,77                 1409   

  ELENA UGOLINI_perc  FEDERICO SERRA_total FEDERICO SERRA_perc  \
0              47,71                    57                2,48   
1    

In [6]:
df_data

,comune,provincia,regione,url,MICHELE DE PASCALE_total,MICHELE DE PASCALE_perc,ELENA UGOLINI_total,ELENA UGOLINI_perc,FEDERICO SERRA_total,FEDERICO SERRA_perc,LUCA TEODORI_total,LUCA TEODORI_perc
0,ALTO RENO TERME,BOLOGNA,EMILIA-ROMAGNA,https://elezioni.interno.gov.it/risultati/2024...,1123,"48,93",1095,"47,71",57,"2,48",20,"0,87"
1,ANZOLA DELL'EMILIA,BOLOGNA,EMILIA-ROMAGNA,https://elezioni.interno.gov.it/risultati/2024...,2992,"65,77",1409,"30,97",91,"2,00",57,"1,25"


In [7]:
df_data['perc_diff'] = round(((df_data['MICHELE DE PASCALE_total'] - df_data['ELENA UGOLINI_total']) / df_data['ELENA UGOLINI_total']) * 100, 2)

In [8]:
df_data['comune'] = df_data['comune'].str.title()
df_data['provincia'] = df_data['provincia'].str.title()
df_data['regione'] = df_data['regione'].str.title()
df_data.head(2)

,comune,provincia,regione,url,MICHELE DE PASCALE_total,MICHELE DE PASCALE_perc,ELENA UGOLINI_total,ELENA UGOLINI_perc,FEDERICO SERRA_total,FEDERICO SERRA_perc,LUCA TEODORI_total,LUCA TEODORI_perc,perc_diff
0,Alto Reno Terme,Bologna,Emilia-Romagna,https://elezioni.interno.gov.it/risultati/2024...,1123,"48,93",1095,"47,71",57,"2,48",20,"0,87",2.56
1,Anzola Dell'Emilia,Bologna,Emilia-Romagna,https://elezioni.interno.gov.it/risultati/2024...,2992,"65,77",1409,"30,97",91,"2,00",57,"1,25",112.35


In [9]:
# Remove rows where 'comune' is either 'Sassofeltrio' or 'Montecopiolo'
df_data = df_data[~df_data['comune'].isin(['Sassofeltrio', 'Montecopiolo'])]

# Use .str.replace() to handle string replacement more safely
df_data['comune'] = df_data['comune'].str.replace("Forli'", "Forlì", regex=False)
df_data['comune'] = df_data['comune'].str.replace("Montescudo - Monte Colombo", "Montescudo-Monte Colombo", regex=False)

In [10]:
df_data.to_csv('../output/risultati_er.csv', index=False, encoding='UTF-8-sig')